In [1]:
import psycopg2
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from shap import Explainer
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
server = "localhost"
database = "db_02"
username = "postgres"
password = "ABcd@12#$"
source = "creditcard_train"

data_train=[]

try:
  conn = psycopg2.connect(dbname=database, user=username, password=password, host=server, port="5432", sslmode='disable')
except psycopg2.Error as e:
  print("Error connecting to PostgreSQL database:", e)
else:
  print("Connection established successfully!")

cursor = conn.cursor()

query =f"SELECT * FROM {source}"
cursor.execute(query)

for i in cursor.fetchall():
  data_train.append(i[1:])

column_names = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class']
data_train=pd.DataFrame(data_train, dtype=float, columns=column_names)
# print(data_train)

Connection established successfully!


In [3]:
# Data Pre-Processing
data_train = data_train.fillna(...)

categorical_features = []

if len(categorical_features) > 0:
  encoder = LabelEncoder()
  for feature in categorical_features:
    data_train[feature] = encoder.fit_transform(data_train[feature])

scaler = StandardScaler()

features_to_scale = data_train.drop("Class", axis=1).columns
data_train_scaled = scaler.fit_transform(data_train[features_to_scale])

X_train = data_train_scaled
y_train = data_train["Class"]

In [4]:
print(X_train.shape, y_train.shape)

(213605, 30) (213605,)


In [5]:
model = Sequential([
    Flatten(input_shape=(30,)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid'),
])

c:\Users\979ar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         3,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,289 (48.00 KB)

 Trainable params: 12,289 (48.00 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall'])
model.fit(X_train, y_train, epochs=8)

Epoch 1/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9984 - loss: 0.0141 - precision: 0.6834 - recall: 0.5931
Epoch 2/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9993 - loss: 0.0035 - precision: 0.8153 - recall: 0.7639
Epoch 3/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9992 - loss: 0.0036 - precision: 0.8352 - recall: 0.7743
Epoch 4/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9994 - loss: 0.0025 - precision: 0.8672 - recall: 0.7661
Epoch 5/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9993 - loss: 0.0029 - precision: 0.8608 - recall: 0.7718
Epoch 6/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9995 - loss: 0.0025 - precision: 0.9074 - recall: 0.7766
Epoch 7/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9995 - loss: 0.0024 - precision: 0.9048 - recall: 0.8026
Epoch 8/8
6676/6676 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9995 - loss: 0.0023 - precision: 0.9067 - recall: 0.7692


In [9]:
y_pred = model.predict(X_train).astype(int)

accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

6676/6676 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step
Accuracy: 0.9982444231174364
Precision: 1.0
Recall: 0.05778894472361809
F1-score: 0.10926365795724466


In [ ]:
# # SHAP Explainer
# explainer = Explainer(model, X_train)
# shap_values = explainer(X_train)

# with open("model_dl.pkl", "wb") as f:
#     pickle.dump(model, f)

# with open("explainer_dl.pkl", "wb") as f:
#     pickle.dump(explainer, f)